In [1]:
def rem_beg(text, prefix):
    if text.startswith(prefix):
        return text[len(prefix):]
    return text

In [2]:
def rem_end(text, suffix):
    if text.endswith(suffix):
        return text[:-len(suffix)]
    return text

In [3]:
def rem_both(text, pattern):
    return rem_suffix(rem_prefix(text,pattern),pattern)

In [4]:
def removeNonAscii(text):
    out = ''
    for c in text:
        if ord(c) in range(128):
            out += c
    return out

In [5]:
# Find out title entitites

In [6]:
import bs4 as bs
import urllib.request
import re
import regex
import sys

In [7]:
sauce = urllib.request.urlopen('https://manpages.ubuntu.com/manpages/bionic/en/man1/gcc-8.1.html')
#sauce = open("D:/KGP/man1/abootimg.1.html", 'rb') 

In [8]:
soup = bs.BeautifulSoup(sauce, 'lxml')

if re.search('<b>DESCRIPTION</b>', str(soup)) == None:
    print('Not a manpage')

In [9]:
name_code = re.search(r'<b> *((NAME)|(Name)) *</b>([\s\S]*?)<pre>([\s\S]*?)</pre>', str(soup))
if name_code == None:
    name_code = soup.title.text
else:
    name_code = name_code.group(5)
name_code

'\n       gcc - GNU project C and C++ compiler\n\n'

In [10]:
title = name_code.replace('\n','').strip()
if (title == 'Ubuntu Manpage:'):
    print('Error if it is a manpage')
    
title = regex.search(r'(.*?)[ ]+([\p{Pd}]|(<b>[\p{Pd}]</b>))', title)
if title == None:
    sys.exit()
title = title.group(1)

title = rem_beg(title, '<b>')
title = rem_end(title, '</b>')
title = title.strip()

print(title)

gcc


In [11]:
all_title = title.split(', ')
all_title

['gcc']

In [12]:
section = '1'
for idx, title in enumerate(all_title):
    all_title[idx] = title + '(' + section + ')'
all_title = set(all_title)
all_title

{'gcc(1)'}

In [13]:
all_title = {removeNonAscii(v) for v in all_title}
all_title

{'gcc(1)'}

In [14]:
# Find all href words/phrases

In [15]:
all_href = set()
for url in soup.find_all('a'):
    if url.text != '' and '\n' not in url.text:
        all_href.add(url.text)
all_href = {removeNonAscii(v) for v in all_href}
print(all_href)

{'gpl', '/usr/lib/gcc/', '/lib64', 'longcall', 'cpp', 'http://gcc.gnu.org/wiki/Visibility', 'Menu', '/dev/stderr', 'assert', 'http://infocenter.arm.com/help/topic/com.arm.doc.ecm0359818/ECM0359818_armv8m_security_extensions_reqs_on_dev_tools_1_0.pdf', 'gdb', 'http://nongnu.org/avr-libc/user-manual/', 'A', 'ld', '/lib32', 'http://gcc.gnu.org/c99status.html', 'http://gcc.gnu.org/onlinedocs/gcc/Contributors.html', 'Launchpad', 'gfdl', '/lib', '/./', '/usr/include', 'dbx', 'Ubuntu Manpage Repository', 'fsf-funding', 'as', '/../', 'Close menu', 'gcc-8_8-20180414-1ubuntu2_amd64', 'http://gcc.gnu.org/projects/mipso64-abi.html', '/usr/lib', 'a', 'http://www.openmp.org/', 'Jump to main content', 'gcov', 'http://gcc.gnu.org/readings.html'}


In [16]:
# Find and store the html code for the description part

In [17]:
desc_code =  re.search('((<b> *DESCRIPTION *</b>)|(<b> *Description *</b>)|(<b> *Detailed *</b> <b> *Description *</b>))([\s\S]*?)<pre>([\s\S]*?)</pre>', str(soup))
desc_code = desc_code.group(6)
desc_code

'\n       When you invoke GCC, it normally does preprocessing, compilation, assembly and linking.\n       The "overall options" allow you to stop this process at an intermediate stage.  For\n       example, the <b>-c</b> option says not to run the linker.  Then the output consists of object\n       files output by the assembler.\n\n       Other options are passed on to one or more stages of processing.  Some options control the\n       preprocessor and others the compiler itself.  Yet other options control the assembler and\n       linker; most of these are not documented here, since you rarely need to use any of them.\n\n       Most of the command-line options that you can use with GCC are useful for C programs; when\n       an option is only useful with another language (usually C++), the explanation says so\n       explicitly.  If the description for a particular option does not mention a source\n       language, you can use that option with all supported languages.\n\n       The us

In [18]:
# Find all boldface words from the description

In [19]:
all_b = set()
for met in re.findall(r'((<b>(.*?)</b>\s?)+)', str(desc_code)):
    word = (''.join(''.join(met[0].split('<b>')).split('</b>'))).replace('\n','').strip()
    if (len(word) > 1):
        all_b.add(word)
all_b = {removeNonAscii(v) for v in all_b}
all_b

{'-L',
 '-W',
 '-Wformat',
 '-c',
 '-d -v',
 '-dv',
 '-f',
 '-ffoo',
 '-fmove-loop-invariants',
 '-fno-foo',
 '-gcc',
 '-gcc-',
 '-l',
 'g++',
 'gcc'}

In [20]:
# Find all italics words from the description

In [21]:
all_i = set()
for met in re.findall(r'((<i>(.*?)</i>\s?)+)', str(desc_code)):
    word = (''.join(''.join(met[0].split('<i>')).split('</i>'))).replace('\n','').strip()
    if (len(word) > 1):
        all_i.add(word) 
all_i = {removeNonAscii(v) for v in all_i}
all_i

set()

In [22]:
# Find manpage mentions 

In [23]:
all_mentions = re.findall(r'[^ \n]+[ \n]+\([0-9]\)', soup.get_text())
all_mentions = {v.replace(' ','').replace('\n','') for v in all_mentions}
all_mentions

{'((aligned(8)',
 'alloc_size(1)',
 'bar(0)',
 'bar(5)',
 'foo(0)',
 'foo(1)',
 'i(1)',
 'j(0)',
 'while(0)'}

In [24]:
# Find all system file mentions

In [25]:
all_sys_files = set()
for match in re.findall(r'([ \n=]|^)((/[^/ ,]{2,})+/?)', str(soup.get_text())):
    all_sys_files.add(match[1].replace('\n',''))
all_sys_files = {removeNonAscii(v) for v in all_sys_files}
all_sys_files

{'/.',
 '/../',
 '/60)',
 '/Library/Frameworks.',
 '/System/Library/Frameworks',
 '/bits/stl',
 '/dev/stderr',
 '/tmp/O2-opts',
 '/tmp/O3-opts',
 '/usr/bin/libtool',
 '/usr/include',
 '/usr/lib',
 '/usr/lib/gcc/',
 '/usr/lib/sched.models',
 '/usr/local/lib/bar.',
 '/usr/local/lib/gcc',
 '/usr/local/lib/gcc/'}

In [26]:
# All caps continuous words

In [27]:
all_caps = set()
for met in re.findall(r'[ \n]+(([A-Z_]{2,}\s)+)', str(soup.get_text())):
    all_caps.add(met[0].strip().replace('\n','')) 
all_caps = {removeNonAscii(v) for v in all_caps}
print(all_caps)

{'XSAVEC', 'MS', 'STS', 'FP', 'SVML', 'DSO', 'SUNPRO_DEPENDENCIES', 'TPF OS', 'OBJC_INCLUDE_PATH', 'SDRAM', 'GCC ABI', 'AIX ABI', 'GCSE', 'CPLUS_INCLUDE_PATH', 'C_INCLUDE_PATH', 'VLIW', 'RSA', 'VSX', 'SRAM', 'ACML', 'ARM', 'DSP', 'MASS', 'TLB', 'ENDDO', 'SSYNC', 'COMPILER_PATH', 'TMPDIR', 'AE', 'HPPA', 'GRAPHITE', 'ISEL', 'VIS', 'DEBUG', 'IDT', 'BUGS', 'MB', 'ARC SIMD', 'NAME', 'AVR', 'XL', 'VRSAVE', 'BMX', 'SETLB', 'SOME_CONDITION_THAT_DOES_NOT_HOLD', 'LC_MESSAGES', 'TS', 'RAM', 'POPCNT', 'RX', 'IBM XL', 'VAES', 'PUSH', 'MAKE', 'MDUC', 'AES', 'VEX', 'ICF', 'GCC_COMPARE_DEBUG', 'PE', 'GB', 'POSIX', 'XY', 'SYSV ABI', 'RDRND', 'MP', 'IEEE', 'MAC', 'EABI', 'PPU', 'ELF', 'GCC', 'ADCX', 'MPX', 'DF', 'FRV', 'COPYRIGHT', 'WPA', 'UMIP', 'GCC ICF', 'ARC EM CPU', 'BSP', 'HSA', 'BWX', 'GCC_EXEC_PREFIX', 'CPU', 'TLS', 'MAX', 'MAC AE', 'WBNOINVD', 'LC_CTYPE', 'PA', 'RTL', 'FOOTNOTES', 'PIC', 'RAX', 'LLVM', 'GAS', 'SOM', 'LSAN_OPTIONS', 'MUL AE', 'SP', 'RX CPU', 'VAX', 'TU', 'FX', 'SYNOPSIS', 'SEE A

In [28]:
# All first letter cap mid sentence words 
# !!!! Currently includes all mid sentence all caps continous words (it is a necessity)

In [29]:
all_first_caps = set()
for met in re.findall(r'([A-Za-z0-9\,][ ]+)(([A-Z]([A-Za-z0-9]|[.-][A-Za-z0-9])+[ ]+)*([A-Z][A-Za-z0-9\.-]+([, \n\.]|$)))', str(soup.get_text())):
    all_first_caps.add(met[1].strip().replace('\n','').strip('.').strip(','))
all_first_caps = {removeNonAscii(v) for v in all_first_caps}
print(all_first_caps)

{'Linux', 'ARM Procedure Call Standard', 'F16C', 'Haswell', 'ARC700', 'Pentium III CPU', 'Dynamic', 'Transactional Memory ABI', 'Objective-C', 'UltraSPARC-III', 'Value Range Propagation', 'Intel MPX', 'V7', 'P0', 'VSX', 'Application Program Interface', 'Inhibit', 'ARM', 'Built-in Functions', 'STATUS32', 'TLB', 'FPU-specific', 'ColdFire V4e', 'Landing CPU', 'HPPA', 'SSE3', 'POWER4', 'MB', 'DLLs', 'Do', 'ARMv8.4-A', 'ARC SIMD', 'MIPS16e ASE', 'SH4-340', 'ARMv7-M', 'VFPv4', 'FCSR Flush', 'NOPs', 'Program', 'VRSAVE', 'Front-Cover', 'SImode', 'UltraSPARC', 'Back-', 'RX', 'Device', 'IBM XL', 'Eden Nehemiah CPU', 'LAC', 'SPARC ABI', 'Intel Pentium III CPU', 'GB', 'MFC', 'Function', 'POSIX', 'Built-in Macros', 'TLBs', 'Mod', 'ARMv6-M', 'Atmel AVR', 'Info', 'Altera Nios II ELF', 'Intel MCU', 'ARC V2', 'Sandy Bridge CPU', 'NaNs', 'GCC ICF', 'ACML ABI-compatible', 'SH3e', 'Back-Cover Text', 'ColdFire', 'AVX512VNNI', 'MacOS', 'All', 'PA', 'AVX512VL', 'RX CPU', 'CR16C', 'FX', 'TU', 'X', 'Standard',

In [30]:
# List of proper nouns

In [31]:
import nltk

In [32]:
sentences = nltk.sent_tokenize(soup.get_text())

sent_words = []
sent_tagged = []

for sentence in sentences:
    words = nltk.word_tokenize(sentence)
    sent_words.append(words)
    sent_tagged.append(nltk.pos_tag(words))

In [33]:
chunkGram = '''Chunk: {(<NNP>(<,>|<CC>))+<NNP>}'''
chunkParser = nltk.RegexpParser(chunkGram)
all_NP_list = set()

for idx, sentence in enumerate(sentences):
    chunked = chunkParser.parse(sent_tagged[idx])
    for tree in chunked.subtrees(filter=lambda t: t.label() == 'Chunk'):
        for ind in range(0, len(tree.leaves()), 2):
            all_NP_list.add(tree.leaves()[ind][0])
all_NP_list = {removeNonAscii(v) for v in all_NP_list}
all_NP_list

{'+0.0',
 '+2',
 '+nofp',
 '+nosimd',
 '-0.0',
 '-Bfoo/',
 '-D',
 '-G',
 '-I',
 '-M',
 '-MM',
 '-Map',
 '-Map=output.map',
 '-O',
 '-O1',
 '-O2',
 '-O3',
 '-Og',
 '-Os',
 '-U',
 '-Werror=',
 '-Wl',
 '-Wno-error=',
 '-Wno-int-to-pointer-cast',
 '-Wno-pointer-to-int-cast',
 '-Wshadow=global',
 '-Wshadow=local',
 '-fPIC',
 '-fPIE',
 '-fapple-kext',
 '-fcommon',
 '-fdebug-prefix-map',
 '-fdump-rtl-btl1',
 '-fdump-rtl-btl2',
 '-fdump-rtl-ce1',
 '-fdump-rtl-ce2',
 '-fdump-rtl-cse1',
 '-fdump-rtl-cse2',
 '-fdump-rtl-dce1',
 '-fdump-rtl-dce2',
 '-fdump-rtl-fwprop1',
 '-fdump-rtl-fwprop2',
 '-fdump-rtl-gcse1',
 '-fdump-rtl-gcse2',
 '-fdump-rtl-sched1',
 '-fdump-rtl-sched2',
 '-fdump-rtl-subreg1',
 '-fdump-rtl-subreg2',
 '-flto',
 '-fmacro-prefix-map',
 '-fno-builtin',
 '-fno-rtti',
 '-fno-signed-zeros',
 '-fno-strict-overflow',
 '-fno-trapping-math',
 '-fno-trapv',
 '-fno-weak',
 '-fpatchable-function-entry=N',
 '-fpeel-loops',
 '-fpic',
 '-fpie',
 '-fsanitize=address',
 '-fsanitize=bounds-stri

In [34]:
# A dictionary that will store the type of each entity
# The dict will have a 9 dim binary vector corresponding to each entity with indices indicating the following:

# 0 title
# 1 href
# 2 bold
# 3 italics
# 4 manpage mention
# 5 system file mention
# 6 All Caps continuous words
# 7 First letter cap mid sentence words
# 8 List of NNP

type_dict = {}

In [35]:
# Add one entity to the type_dict
def add_type(entity, entity_type):
    if entity not in type_dict:
        type_dict[entity] = []
    if entity_type not in type_dict[entity]:
        type_dict[entity].append(entity_type)

In [36]:
# Add an entire section of entities to the type_dict
def add_section(entity_list, section_id):
    for entity in entity_list:
        add_type(entity, section_id)

In [37]:
# Populate the type_dict

add_section(all_title, 0)
add_section(all_href, 1)
add_section(all_b, 2)
add_section(all_i, 3)
add_section(all_mentions, 4)
add_section(all_sys_files, 5)
add_section(all_caps, 6)
add_section(all_first_caps, 7)
add_section(all_NP_list, 8)

type_dict

{'gcc(1)': [0],
 'gpl': [1],
 '/usr/lib/gcc/': [1, 5],
 '/lib64': [1],
 'longcall': [1],
 'cpp': [1],
 'http://gcc.gnu.org/wiki/Visibility': [1],
 'Menu': [1],
 '/dev/stderr': [1, 5],
 'assert': [1],
 'http://infocenter.arm.com/help/topic/com.arm.doc.ecm0359818/ECM0359818_armv8m_security_extensions_reqs_on_dev_tools_1_0.pdf': [1],
 'gdb': [1],
 'http://nongnu.org/avr-libc/user-manual/': [1],
 'A': [1],
 'ld': [1],
 '/lib32': [1],
 'http://gcc.gnu.org/c99status.html': [1],
 'http://gcc.gnu.org/onlinedocs/gcc/Contributors.html': [1],
 'Launchpad': [1, 7],
 'gfdl': [1],
 '/lib': [1],
 '/./': [1],
 '/usr/include': [1, 5],
 'dbx': [1],
 'Ubuntu Manpage Repository': [1, 7],
 'fsf-funding': [1],
 'as': [1],
 '/../': [1, 5],
 'Close menu': [1],
 'gcc-8_8-20180414-1ubuntu2_amd64': [1],
 'http://gcc.gnu.org/projects/mipso64-abi.html': [1],
 '/usr/lib': [1, 5],
 'a': [1],
 'http://www.openmp.org/': [1],
 'Jump to main content': [1],
 'gcov': [1],
 'http://gcc.gnu.org/readings.html': [1],
 '-gcc':

In [38]:
# Build the full entity list

# ???????????????????
# Should grep -e like commands be grep -e(1)

In [39]:
entities = all_title | all_href | all_b | all_i | all_mentions | all_sys_files | all_caps | all_first_caps | all_NP_list

In [40]:
# Remove entities with length 1 or 0
for entity in entities.copy():
    if len(entity) <= 1:
        print(entity)
        entities.discard(entity)

x
=
K
F
a
A
R
@
N
X
G
Z
Y
L
M
C
>
D


In [41]:
# Remove stopwords from entities
from nltk.corpus import stopwords
del_words = stopwords.words("english")

for word in del_words:
    entities.discard(word)
    entities.discard(word[0].upper() + word[1:])

In [42]:
# Change grep (1) to grep(1)

for each in entities.copy():
    if len(each) >= 3 and each[-3] == '(' and each[-2].isdigit() and each[-1] == ')':
        each1 = each.replace(' ','').replace('\n','')
        if each1 != each:
            entities.discard(each)
            entities.add(each1)
            type_dict[each1] = type_dict[each]
            del type_dict[each]
entities

{'-fsanitize=kernel-address',
 'gpl',
 'Linux',
 'SUNPRO_DEPENDENCIES',
 'Dynamic',
 'Objective-C',
 'OBJC_INCLUDE_PATH',
 '-fno-builtin',
 'VSX',
 '-mv850e',
 'TLB',
 'Landing CPU',
 'GRAPHITE',
 'SSE3',
 'MIPS16e ASE',
 'ARMv8.4-A',
 'SH4-340',
 '.hpp',
 'ARMv7-M',
 'VFPv4',
 'NOPs',
 'Program',
 'MCF5203',
 'SImode',
 'Back-',
 'bar(5)',
 'RX',
 'Device',
 'IBM XL',
 'Eden Nehemiah CPU',
 'LAC',
 'SPARC ABI',
 'ICF',
 'Function',
 'TLBs',
 'ARMv6-M',
 'Atmel AVR',
 'Info',
 'Altera Nios II ELF',
 '-fdump-rtl-gcse2',
 'ARC V2',
 'Back-Cover Text',
 'LC_CTYPE',
 'MacOS',
 'http://infocenter.arm.com/help/topic/com.arm.doc.ecm0359818/ECM0359818_armv8m_security_extensions_reqs_on_dev_tools_1_0.pdf',
 'AT697E',
 'Front-Cover Text',
 '-msoft-float',
 'Relaxed Memory Order',
 'Feature Modifiers',
 'Flash',
 'GFNI',
 'Enable Scalable Vector Extension',
 'Ft32B',
 'PowerPC ABI',
 'Floating-point',
 'IT',
 '-mvis',
 'NOP',
 'LC_ALL',
 'Tools Engineering',
 'DIEs',
 'Texts',
 'SCoP',
 'Make',
 

In [43]:
# Remove grep if grep(1) is already present
for each in entities.copy():
    if len(each) >= 3 and each[-3] == '(' and each[-2].isdigit() and each[-1] == ')':
        del_entity = each[:-3]
        entities.discard(del_entity)
entities

{'-fsanitize=kernel-address',
 'gpl',
 'Linux',
 'SUNPRO_DEPENDENCIES',
 'Dynamic',
 'Objective-C',
 'OBJC_INCLUDE_PATH',
 '-fno-builtin',
 'VSX',
 '-mv850e',
 'TLB',
 'Landing CPU',
 'GRAPHITE',
 'SSE3',
 'MIPS16e ASE',
 'ARMv8.4-A',
 'SH4-340',
 '.hpp',
 'ARMv7-M',
 'VFPv4',
 'NOPs',
 'Program',
 'MCF5203',
 'SImode',
 'Back-',
 'bar(5)',
 'RX',
 'Device',
 'IBM XL',
 'Eden Nehemiah CPU',
 'LAC',
 'SPARC ABI',
 'ICF',
 'Function',
 'TLBs',
 'ARMv6-M',
 'Atmel AVR',
 'Info',
 'Altera Nios II ELF',
 '-fdump-rtl-gcse2',
 'ARC V2',
 'Back-Cover Text',
 'LC_CTYPE',
 'MacOS',
 'http://infocenter.arm.com/help/topic/com.arm.doc.ecm0359818/ECM0359818_armv8m_security_extensions_reqs_on_dev_tools_1_0.pdf',
 'AT697E',
 'Front-Cover Text',
 '-msoft-float',
 'Relaxed Memory Order',
 'Feature Modifiers',
 'Flash',
 'GFNI',
 'Enable Scalable Vector Extension',
 'Ft32B',
 'PowerPC ABI',
 'Floating-point',
 'IT',
 '-mvis',
 'NOP',
 'LC_ALL',
 'Tools Engineering',
 'DIEs',
 'Texts',
 'SCoP',
 'Make',
 

In [44]:
# Holling Algo Implementation

In [45]:
def beg_contain(A, a): # Checks whether a is contained in the beginning of A
    
    if len(a) > len(A):
        return False
    
    for i in range(len(a)):
        if a[i] != A[i]:
            return False
    return True

In [46]:
w = 5

import csv

zipf_words = {}

with open('D:\\KGP\\zip_words.txt','r') as csv_file:
    csv_reader = csv.reader(csv_file)
    ind = 0
    for word in csv_reader:
        zipf_words[word[0]] = ind
        ind += 1

zipf_words

{'minimum': 0,
 'common': 1,
 'With': 2,
 'Cholesky': 3,
 'optimization': 4,
 'service': 5,
 'box': 6,
 'Q**H': 7,
 'password': 8,
 'extensions': 9,
 'distribution': 10,
 '25': 11,
 'signal': 12,
 'LDX11': 13,
 'syntax': 14,
 'virtual': 15,
 'declared': 16,
 'final': 17,
 'Format': 18,
 'active': 19,
 'square': 20,
 'copies': 21,
 'estimates': 22,
 'One': 23,
 'targets': 24,
 'initial': 25,
 'certificate': 26,
 'logical': 27,
 'routines': 28,
 'language': 29,
 'model': 30,
 'head': 31,
 'LDX21': 32,
 'successfully': 33,
 'removed': 34,
 'addresses': 35,
 'separate': 36,
 'queue': 37,
 'pattern': 38,
 'Any': 39,
 '”': 40,
 'BERR': 41,
 '-h': 42,
 'U1': 43,
 'sections': 44,
 'longer': 45,
 'U2': 46,
 'XML': 47,
 'destination': 48,
 'JOBV': 49,
 'shell': 50,
 'ICOMPQ': 51,
 'hardware': 52,
 '“': 53,
 '-s': 54,
 'permutation': 55,
 'compilation': 56,
 'now': 57,
 'Science': 58,
 'Q**T': 59,
 'LDH': 60,
 'causes': 61,
 'Function': 62,
 'Documentation': 63,
 'perform': 64,
 'p': 65,
 'record

In [47]:
zipf_vector = {}
for entity in entities:
    zipf_vector[entity] = set()

In [56]:
c = 2

# context_dict[entity] is a list of tuples of the form [context, count] where context is a list and count is the no of 
# occurences of that context
# occ_index_dict has pairs of the form (sentence id, entity start word id) for each entity
# entity_len_dict stores the word len of each entity

context_dict = {}
occ_index_dict = {}
entity_len_dict = {}
context_word_dict = {}

def context_search(entity):
    
    for index, sentence in enumerate(sentences):
        
        comp_ind = 0
        start = 0
        
        words = sent_words[index]
        tagged = sent_tagged[index]

        for idx, word in enumerate(words):
            
            if beg_contain(entity[comp_ind: ], word):
                comp_ind += len(word)
            else:
                comp_ind = 0
                start = idx + 1
        
            if comp_ind == len(entity):
                
                context = ""
                word_context = ""
                
                end = idx
                
                occ_index_dict[orig_entity].append((index, start))
                entity_len_dict[orig_entity] = end - start + 1
            
                for i in range(start - c, 0):
                    context += '$ '
                    word_context += '$ '

                for i in range(max(start - c, 0), start):
                    context += tagged[i][1] + ' '
                    word_context += words[i] + ' '

                context += '@@'
                word_context += '@@'

                for i in range(end + 1, min(len(words) - 1, end + c) + 1):
                    context += ' ' + tagged[i][1]
                    word_context += ' ' + words[i]

                for i in range(len(words), end + c + 1):
                    context += ' $'
                    word_context += ' $'
                
                if context in context_dict[orig_entity]: # if context is already present in the context list, just increment count
                    context_dict[orig_entity][context] += 1
                else: # Add new context
                    context_dict[orig_entity][context] = 1
                
                if word_context in context_word_dict[orig_entity]: # if context is already present in the context list, just increment count
                    context_word_dict[orig_entity][word_context] += 1
                else: # Add new context
                    context_word_dict[orig_entity][word_context] = 1
                    
                # Find the zipf words in +- w window
                
                for z_in in range(max(start - w, 0), min(end + w + 1, len(words))):
                    if words[z_in] in zipf_words:
                        #zipf_vector[entity]
                        #zipf_words[words[z_in]]
                        zipf_vector[orig_entity].add(zipf_words[words[z_in]])


In [57]:
for orig_entity in entities:
    
    entity = orig_entity.replace(' ', '')
    
    context_dict[orig_entity] = {}
    occ_index_dict[orig_entity] = []
    entity_len_dict[orig_entity] = -1
    context_word_dict[orig_entity] = {}
    
    context_search(entity)
    
    if len(entity) >= 3 and entity[-3] == '(' and entity[-1] == ')':  # If orig_entity is for eg grep(1), then after context searching for grep(1), also context search for grep
        entity = entity[:-3]
        context_search(entity) 

In [59]:
zipf_vector['MIPS64']

{112, 939, 951, 1427, 2882, 2894, 12838, 13668}

In [ ]:
# Calculate occurence_count for all entities
occ_count = {}
for entity in entities:
    occ_count[entity] = 0
    for context in context_dict[entity]:
        occ_count[entity] += context_dict[entity][context]
sorted(occ_count.items(), key = lambda kv:(kv[1], kv[0]), reverse = True)

In [ ]:
# Check if any context_count is 0 for any entity(which means an error)
# Also, delete also those from the entity list for now 

# !!!!!!!!!!!!!!! Error Explanations:
# -fdump-tree-pre=/dev/stderr taken as one word by nltk
#  ../lib64, ../lib, ../lib32 are words but only the part after .. is a hyperlink. So href extraction extracts only after ..
# machine-gcc-version is one word but only the -gcc- is bold. So only it is extracted as an entity
# -v. taken as one word by nltk
# OK. taken as one word by nltk

for entity in entities.copy():
    if occ_count[entity] == 0:
        print(entity)
        entities.discard(entity)

In [ ]:
print(entities)

In [ ]:
for entity in entities:
    context_dict[entity] = {removeNonAscii(u):v for u,v in context_dict[entity].items()}
context_dict

In [ ]:
# Find out which entities occur in a +- k window (but in the same sentence) for all entities

In [ ]:
def co_occur(l1, len1, l2, len2, k):
    ind1 = 0
    ind2 = 0
    
    while ind1 < len(l1) and ind2 < len(l2):
        if l1[ind1][0] < l2[ind2][0]:
            ind1 += 1
        elif l1[ind1][0] > l2[ind2][0]:
            ind2 += 1
        else:
            if l1[ind1][1] + len1 - 1 + k < l2[ind2][1]:
                ind1 += 1
            elif l1[ind1][1] > l2[ind2][1] + len2 - 1 + k:
                ind2 += 1 
            else:
                return True
    return False

In [ ]:
k = 2
co_occ_dict = {} # co_occ_dict[entity] stores the entities that co-occur with entity

res_set = entities.copy()

for entity in entities:
    co_occ_dict[entity] = []

while len(res_set) > 0:
    entity1 = res_set.pop()
    for entity2 in res_set:
        if co_occur(occ_index_dict[entity1], entity_len_dict[entity1], occ_index_dict[entity2], entity_len_dict[entity2], k):
            co_occ_dict[entity1].append(entity2)
            co_occ_dict[entity2].append(entity1)
        

In [ ]:
co_occ_dict['Launchpad']

In [ ]:
# Knowledge Graph Construction : Non anchor edges

In [ ]:
import networkx as nx

In [ ]:
threshold = 1 # 2 entities will be connected by an edge if no of common contexts > threshold * no of contexts for both entities

G = nx.Graph()

rem_set = entities.copy()

while len(rem_set) > 0:
    
    entity1 = rem_set.pop()
    
    for entity2 in rem_set:
        
        n1 = 0 # No of common contexts of entity1
        n2 = 0 # No of common contexts of entity2
        
        for context1 in context_dict[entity1]:
            for context2 in context_dict[entity2]:
                if context1 == context2:
                    n1 += context_dict[entity1][context1]
                    n2 += context_dict[entity2][context2]
                    break
            
        
        if (n1 >= threshold * occ_count[entity1]) and (n2 >= threshold * occ_count[entity2]):
            G.add_edge(entity1, entity2)

In [ ]:
nx.info(G)

In [ ]:
G.nodes()

In [ ]:
print(G.adj['Sandy Bridge CPU'])
print(context_dict['Sandy Bridge CPU'])
print(context_dict['Broadwell CPU'])

In [ ]:
# Knowledge Graph Construction : Anchor edges

In [ ]:
# Find the 'SEE ALSO' section's html code 
see_also_code = re.search(r'<b>SEE</b> <b>ALSO</b>([\s\S]*?)<pre>([\s\S]*?)</pre>', str(soup)).group(2)
print(see_also_code)

In [ ]:
# Find the entities corresponding to href tags
anchor_entities = [each[1] + each[2] for each in re.findall(r'<a(.*?)>(.*?)</a></u>(\([0-9]\))', see_also_code)]
anchor_entities

In [ ]:
# Find the other underlined entities
anchor_entities += re.findall(r'<u>([^<>]*)</u>', see_also_code)
anchor_entities

In [ ]:
# Remove duplicates
anchor_entities = set(anchor_entities)
anchor_entities

In [ ]:
# Remove entity if entity(id) is already present
for each in anchor_entities.copy():
    if len(each) >= 3 and each[-3] == '(' and each[-1] == ')':
        anchor_entities.discard(each[:-3])
anchor_entities = {removeNonAscii(v) for v in anchor_entities}
anchor_entities

In [ ]:
G_anchor = nx.Graph()

for entity1 in all_title:
    for entity2 in anchor_entities:
        G_anchor.add_edge(entity1, entity2)

import matplotlib.pyplot as plt
print(nx.info(G_anchor))
nx.draw(G_anchor, with_labels = True)
plt.show()

In [ ]:
# Write the extractions of this manpage to a file

In [ ]:
import csv

In [ ]:
with open('C:\\Users\\shash\\Desktop\\props.txt', 'w', newline='') as file:
    fieldnames = ['name','type','context_dict','co_occ', 'zipf_vector']
    csv_writer = csv.DictWriter(file, fieldnames=fieldnames, delimiter = '\t')
    csv_writer.writeheader()
    for entity in entities:
        new_dict = {}
        new_dict['name'] = entity
        new_dict['type'] = type_dict[entity]
        new_dict['context_dict'] = context_dict[entity]
        new_dict['co_occ'] = co_occ_dict[entity]
        new_dict['zipf_vector'] = zipf_vector[entity]
        csv_writer.writerow(new_dict)

In [ ]:
with open('C:\\Users\\shash\\Desktop\\anchor_links.txt', 'w', newline='') as file:
    fieldnames = ['title','links']
    csv_writer = csv.DictWriter(file, fieldnames=fieldnames, delimiter = '\t')
    csv_writer.writeheader()
    
    for title in all_title:
        new_dict = {}
        new_dict['title'] = title
        new_dict['links'] = anchor_entities
        csv_writer.writerow(new_dict)

In [ ]:
import os
os.listdir('D:\\KGP\\man2\\')